In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd

## Saving and loading models, with application to the EuroSat dataset

#### The EuroSAT dataset

![euro](Datasets/EuroSAT/eurosat_overview.jpg)

In this assignment, you will use the [EuroSAT dataset](https://github.com/phelber/EuroSAT). It consists of 27000 labelled Sentinel-2 satellite images of different land uses: residential, industrial, highway, river, forest, pasture, herbaceous vegetation, annual crop, permanent crop and sea/lake. For a reference, see the following papers:
- Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
- Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.

Your goal is to construct a neural network that classifies a satellite image into one of these 10 classes, as well as applying some of the saving and loading techniques you have learned in the previous sessions.

In [17]:
### Initiate an instnace of ImagaDataGenerator ###
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.,validation_split=0.2)

path = "Datasets\EuroSAT\EuroSAT_data"

train_dataset = image_generator.flow_from_directory(batch_size=64,
                                                 directory=path,
                                                 shuffle=True,
                                                 target_size=(64, 64), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=64,
                                                 directory=path,
                                                 shuffle=True,
                                                 target_size=(64, 64), 
                                                 subset="validation",
                                                 class_mode='categorical')


Found 21600 images belonging to 10 classes.
Found 5400 images belonging to 10 classes.


#### Build the neural network model
You can now construct a model to fit to the data. Using the Sequential API, build your model according to the following specifications:

* The model should use the input_shape in the function argument to set the input size in the first layer.
* The first layer should be a Conv2D layer with 16 filters, a 3x3 kernel size, a ReLU activation function and 'SAME' padding. Name this layer 'conv_1'.
* The second layer should also be a Conv2D layer with 8 filters, a 3x3 kernel size, a ReLU activation function and 'SAME' padding. Name this layer 'conv_2'.
* The third layer should be a MaxPooling2D layer with a pooling window size of 8x8. Name this layer 'pool_1'.
* The fourth layer should be a Flatten layer, named 'flatten'.
* The fifth layer should be a Dense layer with 32 units, a ReLU activation. Name this layer 'dense_1'.
* The sixth and final layer should be a Dense layer with 10 units and softmax activation. Name this layer 'dense_2'.

In total, the network should have 6 layers.

In [33]:
val_steps = validation_dataset.n // validation_dataset.batch_size
train_steps = train_dataset.n // train_dataset.batch_size
print(f"Train steps per epoch: {train_steps}")
print(f"Validation steps per epoch: {val_steps}")


Train steps per epoch: 337
Validation steps per epoch: 84


What does step per epoch mean? It means the number of batches that makes up an epoch. For instant 
for the validation set there are 83 batches of 64 photos  and 1 batch of 88 photos that is 84 batches in total.

In [29]:
def get_new_model():
    """
    This function should build a Sequential model according to the above specification. Ensure the 
    weights are initialised by providing the input_shape argument in the first layer, given by the
    function argument.
    Your function should also compile the model with the Adam optimiser, sparse categorical cross
    entropy loss function, and a single accuracy metric.
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=((64, 64, 3))))
    model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",padding="same",name="conv_1"))
    model.add(tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),activation="relu",padding="same",name="conv_2"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(8,8),name="pool_1"))
    
    model.add(tf.keras.layers.Flatten(name="flatten"))
    model.add(tf.keras.layers.Dense(units=32,activation="relu",name="dense_1"))
    model.add(tf.keras.layers.Dense(units=10,activation="softmax",name="dense_2"))
    
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model 

def get_test_accuracy(model,data,steps,**kwargs):
    """Test model classification accuracy"""
    test_loss, test_acc = model.evaluate_generator(data,steps,**kwargs)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))



In [30]:
model = get_new_model()
model.summary()
get_test_accuracy(model,validation_dataset,val_steps)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 8)         1160      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 8, 8, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                16416     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 18,354
Trainable params: 18,354
Non-trainable params: 0
_________________________________________________

#### Create checkpoints to save model during training, with a criterion

You will now create three callbacks:
- `checkpoint_every_epoch`: checkpoint that saves the model weights every epoch during training
- `checkpoint_best_only`: checkpoint that saves only the weights with the highest validation accuracy. Use the testing data as the validation data.
- `early_stopping`: early stopping object that ends training if the validation accuracy has not improved in 3 epochs.

In [44]:
def get_checkpoint_every_epoch():
    """
    This function should return a ModelCheckpoint object that:
    - saves the weights only at the end of every epoch
    - saves into a directory called 'checkpoints_every_epoch' inside the current working directory
    - generates filenames in that directory like 'checkpoint_XXX' where
      XXX is the epoch number formatted to have three digits, e.g. 001, 002, 003, etc.
    """
    path = 'Datasets/EuroSAT/checkpoints_every_epoch/checkpoint_{epoch:02d}' # path to save model
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=path, frequency='epoch',save_weights_only=True,verbose=2)
    return checkpoint

    

def get_checkpoint_best_only():
    """
    This function should return a ModelCheckpoint object that:
    - saves only the weights that generate the highest validation (testing) accuracy
    - saves into a directory called 'checkpoints_best_only' inside the current working directory
    - generates a file called 'checkpoints_best_only/checkpoint'
    """
    path = 'Datasets/EuroSAT/checkpoints_best_only/checkpoint'# path to save model
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=path, save_best_only=True,save_weights_only=True,verbose=2)
    return checkpoint

def get_early_stopping():
    """
    This function should return an EarlyStopping callback that stops training when
    the validation (testing) accuracy has not improved in the last 3 epochs.
    HINT: use the EarlyStopping callback with the correct 'monitor' and 'patience'
    """
    return tf.keras.callbacks.EarlyStopping(monitor='accuracy',min_delta=0.01,patience=3,mode="max")
    
    
    

#### Train model using the callbacks

Now, you will train the model using the three callbacks you created. If you created the callbacks correctly, three things should happen:
- At the end of every epoch, the model weights are saved into a directory called `checkpoints_every_epoch`
- At the end of every epoch, the model weights are saved into a directory called `checkpoints_best_only` **only** if those weights lead to the highest test accuracy
- Training stops when the testing accuracy has not improved in three epochs.

You should then have two directories:
- A directory called `checkpoints_every_epoch` containing filenames that include `checkpoint_001`, `checkpoint_002`, etc with the `001`, `002` corresponding to the epoch
- A directory called `checkpoints_best_only` containing filenames that include `checkpoint`, which contain only the weights leading to the highest testing accuracy

In [50]:
# Run this cell to create the callbacks

checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()


# Train model using the callbacks you just created

callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
history = model.fit(train_dataset,steps_per_epoch=train_steps,epochs=2,callbacks=callbacks)

Epoch 1/2
337/337 [==============================] - ETA: 0s - loss: 0.6774 - accuracy: 0.7552
Epoch 00001: saving model to Datasets/EuroSAT/checkpoints_every_epoch/checkpoint_01
337/337 [==============================] - 71s 210ms/step - loss: 0.6774 - accuracy: 0.7552
Epoch 2/2
337/337 [==============================] - ETA: 0s - loss: 0.6518 - accuracy: 0.7671
Epoch 00002: saving model to Datasets/EuroSAT/checkpoints_every_epoch/checkpoint_02
337/337 [==============================] - 65s 192ms/step - loss: 0.6518 - accuracy: 0.7671


#### Create new instance of model and load on both sets of weights

Now you will use the weights you just saved in a fresh model. You should create two functions, both of which take a freshly instantiated model instance:
- `model_last_epoch` should contain the weights from the latest saved epoch
- `model_best_epoch` should contain the weights from the saved epoch with the highest testing accuracy

_Hint: use the_ `tf.train.latest_checkpoint` _function to get the filename of the latest saved checkpoint file. Check the docs_ [_here_](https://www.tensorflow.org/api_docs/python/tf/train/latest_checkpoint).

In [51]:
def get_model_last_epoch(model):
    """
    This function should create a new instance of the CNN you created earlier,
    load on the weights from the last training epoch, and return this model.
    """
   
    path = tf.train.latest_checkpoint(checkpoint_dir="Datasets/EuroSAT/checkpoints_every_epoch")
    model.load_weights(filepath= path)
    return model
    
    
    
### THIS CALLNACK IS ONLy CALLED WHEN WE HAVE A VALIDATION SPLIT ###
def get_model_best_epoch(model):
    """
    This function should create a new instance of the CNN you created earlier, load 
    on the weights leading to the highest validation accuracy, and return this model.
    """
    path = "Datasets/EuroSAT/checkpoints_best_only/checkpoint" # path to save model
    model.load_weights(filepath= path)
    return model 
    

In [52]:
model_last_epoch = get_model_last_epoch(get_new_model())
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch,validation_dataset,val_steps)


Model with last epoch weights:
accuracy: 0.771


#### Load, from scratch, a model trained on the EuroSat dataset.

In your workspace, you will find another model trained on the `EuroSAT` dataset in `.h5` format. This model is trained on a larger subset of the EuroSAT dataset and has a more complex architecture. The path to the model is `models/EuroSatNet.h5`. See how its testing accuracy compares to your model!

In [62]:
def get_model_eurosatnet():
    """
    This function should return the pretrained EuroSatNet.h5 model.
    """

    path = "Datasets\EuroSAT\EuroSatNet.h5" # path to save model
    model = tf.keras.models.load_model(filepath=path)
    return model
    

In [63]:
Euro = get_model_eurosatnet()
Euro.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 16)        6416      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 32, 32, 16)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 32, 32, 16)        2320      
_________________________________________________________________
conv_4 (Conv2D)              (None, 32, 32, 16)        6416      
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 16, 16, 16)        0         
_________________________________________________________________
conv_5 (Conv2D)              (None, 16, 16, 16)      